# Service Shop Reports Tagger
This Notebook requires SSR_Spacy Model - which can be created using "SSR_spacy_model" notebook

## Steps
1. Load Data
2. Clean Text
3. Load spacy model, transform clean text into spacy docs
4. Get file tags

In [1]:
# !pip install spacy
# !pip install 
# !pip install matplotlib
# !pip install seaborn
# !pip install fitz
# !pip install PyMuPDF

In [3]:
!pip install PyPDF2

     -------------------------------------- 222.8/222.8 kB 2.3 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import fitz
import os
import glob

import re

import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from spacy import displacy
# from spacy.language import Language
# from spacy_langdetect import LanguageDetector


from PyPDF2 import PdfFileReader, PdfFileWriter
import io

from collections import Counter
import shutil

from itertools import chain

import hashlib

In [7]:
import warnings
warnings.filterwarnings("ignore")

# 1. Load Data
- only pdf files in location

In [8]:
def load_single_file(file):
    '''
    Input: 
    - pdf file name/path
    Return: 
    - string, extracted text from all pages
    '''
    with fitz.open(file) as doc:
        text = ""
        for page in doc:
#             text += page.getText()            # get_text
            text += page.get_text() 
    return text



def load_multiple_files(folder_name):
    '''
    Input: 
    - folder_name with pdf files. Folder must be in current working directory
    Return: 
    - 2 lists:list of string, extracted text from all pages and list of file names
    '''
    texts = []
    file_names = []
    cur_dir = os.getcwd()
    path = f'{cur_dir}\\{folder_name}'
    os.chdir(path)
    file_list = os.listdir()
    for file in file_list:
        with fitz.open(file) as doc:
            text = ""
            for page in doc:
#                 text += page.getText()      # get_text
                text += page.get_text()
        texts.append(text)
        file_names.append(file)
    os.chdir(cur_dir)
    return texts, file_names

# 2. Clean Text

In [9]:
def clean_text(text):
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = text.replace('_', ' ')
    text = text.replace('•', ' ')
    text = text.replace(' | ', ' ')
    text = text.encode("ascii", "ignore").decode()
    text = text.lower().strip()
    while "  " in text:
        text = text.replace("  ", " ")
    return text

# 3. Getting tags using spaCy
## 3.1. Serial Numbers

In [10]:
# OEM SNs

def get_sn_oem(doc):
    
    uniq_sns = []
    for token in doc.ents:
        if token.label_ == 'SN_OEM':
            uniq_sns.append(token.text) 
    return set(uniq_sns)


# exclude short sns
def get_sn_oem(doc):
    

    uniq_sns = []
    for token in doc.ents:
        if token.label_ == 'SN_OEM' and len(token.text) > 3:
            uniq_sns.append(token.text) 
    return set(uniq_sns)

In [11]:
# EQ SNs
def get_sn_eq(doc):
    
    uniq_sns = []
    for token in doc.ents:
        if token.label_ == 'SN_EQ':
            uniq_sns.append(token.text) 
    return set(uniq_sns)

In [12]:
# this is classic approach, not using spacy but classic regex

def get_sn_classic(text):
    
    #dzialajace
    sns = []
    pattern_1 = r'serial\snumber:?\s[a-z]{0,3}[0-9]+[a-z]?'
    pattern_2 = r's/?n:?\s[a-z]{0,3}[0-9]+[a-z]?'
    pattern_3 = r'matr.\smacchina:?\s[a-z]{0,3}[0-9]+[a-z]?'
    pattern_4 = r'matricola\smacchina:?\s[a-z]{0,3}[0-9]+[a-z]?'
    
    # urozmaicam
    pattern_1 = r'serial\snumber:?\s[a-z]{0,4}[#-]?[0-9]+[a-z]?/?[0-9]?'
    pattern_2 = r's/?n:?\s[a-z]{0,4}[#-]?[0-9]+[a-z]?/?[0-9]?'
    pattern_3 = r'matr.\smacchina:?\s[a-z]{0,4}[#-]?[0-9]+[a-z]?/?[0-9]?'
    pattern_4 = r'matricola\smacchina:?\s[a-z]{0,4}[#-]?[0-9]+[a-z]?/?[0-9]?'

        # urozmaicam
    pattern_1 = r'serial\snumber:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    pattern_2 = r's/?n:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_3 = r'matr.\smacchina:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_4 = r'matricola\smacchina:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_4 = r'matricola\smacchina:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_5 = r'matricola:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_6 = r'serial\snumber\smacchina:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    pattern_7 = r's/?n\smacchina:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    pattern_8 = r'serial\sn:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    pattern_9 = r'machine\ss\.\sn\.\s:?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    
            # urozmaicam z .
    pattern_1 = r'serial\snumber[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    pattern_2 = r's/?n[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_3 = r'matr.\smacchina[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_4 = r'matricola\smacchina[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_4 = r'matricola\smacchina[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_5 = r'matricola[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]?'
    pattern_6 = r'serial\snumber\smacchina[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    pattern_7 = r's/?n\smacchina[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    pattern_8 = r'serial\sn[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    pattern_9 = r'machine\ss\.\sn\.\s[\.:]?\s[a-z0-9]{0,2}[-]?[a-z0-9]{0,4}[#-]?[0-9]+[a-z]{0,3}/?[0-9]{0,2}'
    
    
    match_1 = re.findall(pattern_1, text)
    match_2 = re.findall(pattern_2, text)
    match_3 = re.findall(pattern_3, text)
    match_4 = re.findall(pattern_4, text)
    match_5 = re.findall(pattern_5, text)
    match_6 = re.findall(pattern_6, text)
    match_7 = re.findall(pattern_7, text)
    match_8 = re.findall(pattern_8, text)
    match_9 = re.findall(pattern_9, text)
    
    sns = match_1 + match_2 + match_3 + match_4 + match_5 + match_6 + match_7 + match_8 + match_9
    
    sns_clean = []
    if len(sns) > 0:
        for x in sns:
            # bierzemy wszystko po dwukropku, a ajk nie ma dwukropka to bierzemy split spacja i [-1] - co jesli 2 czlony?
            sn = x.split(' ')[-1]
            sns_clean.append(sn)
    
    letters = ['g', 'a', 'v', 'c']
    # Typo i mozliwe bledy
    sns_clean_2 = []
    for x in sns_clean:
        if x[:2] == 'go':
            x = x.replace('go', 'g0')
        elif x[:2] == 'vo':
            x = x.replace('vo', 'v0')
        elif x[:2] == 'co':
            x = x.replace('co', 'c0')
        elif x[:2] == 'to':
            x = x.replace('to', 't0')
        
        if x[0] in letters and len(x) == 5 and x[1:].isdigit():
            x = x.replace(x[0], x[0] + '0')
        
        if x[0] == 't' and len(x) == 5:
            x = x.replace('t', 'tv')
        
        if '/' in  x:
            sep_nr = []
            sep_nr = x.split('/')
            x_1 = sep_nr[0]
            if len(sep_nr[1]) <= 2:
                end = -len(sep_nr[1])
                x_2 = sep_nr[0][:end] + sep_nr[1]
            else:
                x_2 = sep_nr[1]
            sns_clean_2.append(x_1)
            sns_clean_2.append(x_2)
        sns_clean_2.append(x)
    
    sns_clean_3 = []
    for x in sns_clean_2:
        sns_clean_3.append(x.replace('o', '0'))
    
    
    
    return set(sns_clean_3)

## 3.2. Report Dates

In [13]:
def get_report_date(doc):

    uniq_date = []
    date = ""
    for token in doc.ents:
        if token.label_ == 'REP_DATE':
            uniq_date.append(token.text) 
#     if len(uniq_date) > 0:
#         date = uniq_date[0].split()[1:]
#     return date
    return uniq_date

In [14]:
def get_report_date_2(doc):

    uniq_date = []
    date = ""
    for token in doc.ents:
        if token.label_ == 'REP_DATE_2':
            uniq_date.append(token.text) 
#     if len(uniq_date) > 0:
#         date = uniq_date[0].split()[1:]
#     return date
    return uniq_date

In [15]:
def get_date(doc):

    uniq_date = []
    for token in doc.ents:
        if token.label_ == 'DATE':
            uniq_date.append(token.text)
            
            if 'copyright' in token.text:
                uniq_date.remove(token.text)
            if 'fm' in token.text:
                uniq_date.remove(token.text)
    return uniq_date

## 3.3. Job Numbers

In [16]:
def get_job(doc):

    jobs = []
    for token in doc.ents:
        if token.label_ == 'JOB #':
            jobs.append(token.text) 
    return jobs

# full
def get_job(doc):

    jobs = []
    job = ''
    for token in doc.ents:
        if token.label_ == 'JOB #':
            jobs.append(token.text)
    if len(jobs) > 0:
        job = jobs[0].split()[-1]
    return job

In [17]:
# dziala na &
def get_job(doc):

    jobs = []
    job = ''
    for token in doc.ents:
        if token.label_ == 'JOB #':
            jobs.append(token.text)
    if len(jobs) > 0:
        job = jobs[0].split()[-1]
        
        # Number:42833i472 
        if ':' in job:
            job = job.split(':')[-1]
            
        # '2302834,5'
        if ',' in job:
                str_end = job.split(',')[1]
                if len(str_end) <= 2:
                    first_number = job.split(',')[0]
                    new_number = first_number[:-len(str_end)] + str_end
                    job = [first_number, new_number]
                    
                #2302834,5248727    
                else:
                    first_number = job.split(',')[0]
                    new_number = job.split(',')[1]
                    job = [first_number, new_number]
        
        # 2302641&2302642
        if '&' in  job:
            first_number = job.split('&')[0]
            new_number = job.split('&')[1]
            job = [first_number, new_number]
        
        # '2302834,5'
        if '/' in job:
                str_end = job.split('/')[1]
                if len(str_end) <= 2:
                    first_number = job.split('/')[0]
                    new_number = first_number[:-len(str_end)] + str_end
                    job = [first_number, new_number]
                    
                #2302834,5248727    
                else:
                    first_number = job.split('/')[0]
                    new_number = job.split('/')[1]
                    job = [first_number, new_number]
        if '#' in job:
            job = job.split('#')[-1]
                    
    return job

## 3.4. Type of report

In [18]:
def get_report_type(doc):

    report = []
    for token in doc.ents:
        if token.label_ == 'REPORT':
            report.append(token.text)
    return report

## * Count unique words

In [19]:
def unique_words(text):
#     return len(set([x for x in text.replace('.','').split(' ') if x.isalpha()]))   # return unique WORDS
    return len(set([x for x in text.replace('.','').split(' ') if x.isalnum()]))  # return unique WORDS and NUMBERS

# Reading files from multiple sources
- Input data is stored on the blade\
\\Bhifdnbj4j\Shared\File_tagger\Input_Data

In [17]:
os.chdir('C:\\Users\\ebryaga\\Desktop\\Data_lake\\1_File_tagger\\Notebooks\\Doc_cluster')

In [ ]:
\\Bhifdnbj4j\Shared\File_tagger\Input_Data

## a. ROP without OCR

In [27]:
# Full
# texts, files = load_multiple_files('No_scan')
# df = pd.DataFrame({'name':files, 'text': texts})

In [28]:
# df.to_csv('ROP_saved.csv')

In [24]:
df = pd.read_csv(r'\\Bhifdnbj4j\Shared\File_tagger\Input_Data\ROP_saved.csv', index_col=0)

In [25]:
df_rop = df[df['name'].str.contains('OCR_') == False]
df_rop['source'] = 'ROP'

## b. OCRed files using Google Vison

In [26]:
df_gv = pd.read_csv(r'\\Bhifdnbj4j\Shared\File_tagger\Input_Data\23_05_google_vision.csv', index_col=0)
df_gv['source'] = 'ROP GV'

## c. Coll Docu

In [27]:
df_cd = pd.read_csv(r'\\Bhifdnbj4j\Shared\File_tagger\Input_Data\OCR_17_coll_docu.csv', index_col=0)
df_cd['source'] = 'Coll Docu 17 CSA GT'

## d. QSC Disk

In [28]:
# Part 1
df_qsc1 = pd.read_csv(r'\\Bhifdnbj4j\Shared\File_tagger\Input_Data\QSC_pdf_1.csv', index_col=0)
df_qsc1['source'] = 'QSC'

# Part 2
df_qsc2 = pd.read_csv(r'\\Bhifdnbj4j\Shared\File_tagger\Input_Data\QSC_pdf_2.csv', index_col=0)
df_qsc2['source'] = 'QSC'

## e. MERGE all sources into one data frame

In [34]:
df = df_rop.append(df_gv).append(df_cd).append(df_qsc1).append(df_qsc2)

In [35]:
df = df[df['text'].isna() == False]

### Clean text

In [36]:
df['clean_text'] = df['text'].apply(clean_text)

### Load spacy model

In [38]:
os.chdir(r'C:\Users\ebryaga\OneDrive - Baker Hughes\File_Tagger\Notebooks\F_Tagger')

In [39]:
nlp = spacy.load('11_05_ssr_ner')

### Create doc object based on clean text

In [40]:
clean_text = df['clean_text'].to_list()
docs = nlp.pipe(clean_text)
docs = list(docs)
df['clean_text_doc'] = docs

### Retrive tags from text using spacy (SNs classic using regex)

In [44]:
df['SN_OEM'] = df['clean_text_doc'].apply(get_sn_oem)
df['SN_EQ'] = df['clean_text_doc'].apply(get_sn_eq)
df['SNs classic'] = df['clean_text'].apply(get_sn_classic)
df['report_date'] = df['clean_text_doc'].apply(get_report_date)
df['report_date_2'] = df['clean_text_doc'].apply(get_report_date_2)
df['date'] = df['clean_text_doc'].apply(get_date)
df['job'] = df['clean_text_doc'].apply(get_job)
# df['language'] = df['clean_text_doc'].apply(check_sent_lang) #slow, checks if it has any sentance in italian
df['report'] = df['clean_text_doc'].apply(get_report_type)
df['unique_words'] = df['clean_text'].apply(unique_words)

### Serial Number post-processing

In [45]:
def clean_typos(x):
    x = x.replace('o', '0')
    return x

In [46]:
checked_rows = []
for i, row in df.iterrows():
    
    rows_eq= row['SNs classic'].intersection(row['SN_EQ'])
    rows_oem= row['SNs classic'].intersection(row['SN_OEM'])
    checked_rows.append(rows_eq.union(rows_oem))

df['SN_checked'] = checked_rows

In [47]:
checked_rows_2 = []
for i, row in df.iterrows():
    if len(row['SN_checked']) == 0:
        rows_eq = row['SN_EQ'].union(row['SN_OEM'])
        checked_rows_2.append(rows_eq)
    else:
        checked_rows_2.append({})

df['SN_checked_2'] = checked_rows_2

In [48]:
checked_eq = []
for i, row in df.iterrows():

    rows_eq= row['SNs classic'].intersection(row['SN_EQ'])
    checked_eq.append(rows_eq)


df['SN_checked_eq'] = checked_eq

In [49]:
checked_oem = []
for i, row in df.iterrows():

    rows_eq= row['SNs classic'].intersection(row['SN_OEM'])
    checked_oem.append(rows_eq)


df['SN_checked_oem'] = checked_oem

In [50]:
final_sn = []
for i, row in df.iterrows():
    rows_eq = row['SN_checked'].union(row['SN_checked_2'])
    final_sn.append(rows_eq)

df['SN_checked_final'] = final_sn

### Report type post-processing - standarization

In [51]:
df['report'] = df['report'].apply(lambda x: [i.replace('//00', '').replace('//1','') for i in x])

In [52]:
def report_std(x):
    rep_std = ''
    if len(x) > 0:
        if 'condition' in x[0]:
            rep_std = 'Condition Report'
        if 'final' in x[0]:
            rep_std = 'Final Report'
        if 'preliminar' in x[0]:
            rep_std = 'Preliminary Report'
        if 'inspection' in x[0]:
            rep_std = 'Inspection Report'
    else:
        rep_std = 'Other'
    return rep_std

In [53]:
df['report_std'] = df['report'].apply(report_std)

In [54]:
df['report_std_short'] = df['report_std'].apply(lambda x: x.split(' ')[0])

In [55]:
df['report_std_short'].value_counts()

Final          8654
Preliminary    4378
Condition      2907
Other          1565
Inspection       54
Name: report_std_short, dtype: int64

### Date standarization

In [56]:
final_dates = []

for i, row in df.iterrows():
    if len(row['report_date']) > 0:
        final_dates.append(row['report_date'][0])
    elif len(row['report_date_2']) > 0:
        final_dates.append(row['report_date_2'][0])
    elif len(row['date']) > 0:
        final_dates.append(row['date'][0])
    else:
        final_dates.append('')

In [57]:
df['final_dates'] = final_dates

In [58]:
# add ':' 

def final_dates_update(x):
    return x.replace('data ', 'data: ').replace('date ', 'date: ')

In [59]:
df['final_dates'] = df['final_dates'].apply(final_dates_update)

In [60]:
# add ':' 

def final_dates_update_2(x):
    if ':' in x:    
        return x.split(':')[-1].strip()
    else:
        return x.strip()

In [61]:
df['final_dates'] = df['final_dates'].apply(final_dates_update_2)

In [62]:
def final_dates_transform(x):
    x = x.replace('/', ' ').replace(', ',' ').replace('.',' ').replace('-',' ')
    x = x.replace('  ',' ').replace('  ',' ').replace('  ',' ')
    return x.split(' ')

In [63]:
df['final_dates_test'] = df['final_dates'].apply(final_dates_transform)

In [64]:
df['len_final_dates_test'] = df['final_dates_test'].apply(lambda x: len(x))

In [65]:
def get_day_and_month_year(x):
    day = ''
    month = ''
    year = ''
    if len(x) == 3:
        if len(x[0]) == 4 and x[0].isdigit() == True:
            year = x[0]
            month = x[1]
            day = x[2]
        else:
            year = x[-1]

            # miesiac slownie:
            if x[1].isalpha() == True:
                day = x[0]
                month = x[1]

            elif x[0].isalpha() == True:
                day = x[1]
                month = x[0]

            # wieksza liczba od 12:
            elif x[0].isdigit() and int(x[0]) > 12:
                day = x[0]
                month = x[1]

               # wieksza liczba od 12:
            elif x[1].isdigit() and int(x[1]) > 12:
                day = x[1]
                month = x[0]

                
                #defaultowe wartosci
#             else:
#                 day = x[0]
#                 month = x[1]
        
    return day, month, year

In [66]:
# this method include that in US month is first
index_houston = []
my_dates = []
for i, row in df.iterrows():
    x = row['final_dates_test']
    day = ''
    month = ''
    year = ''
    if len(x) == 3:
        if len(x[0]) == 4 and x[0].isdigit() == True:
            year = x[0]
            month = x[1]
            day = x[2]
        else:
            year = x[-1]

            # miesiac slownie:
            if x[1].isalpha() == True:
                day = x[0]
                month = x[1]

            elif x[0].isalpha() == True:
                day = x[1]
                month = x[0]

            # wieksza liczba od 12:
            elif x[0].isdigit() and int(x[0]) > 12:
                day = x[0]
                month = x[1]

               # wieksza liczba od 12:
            elif x[1].isdigit() and int(x[1]) > 12:
                day = x[1]
                month = x[0]

                
                #defaultowe wartosci
            else:
                if 'houston' in row['clean_text']:
#                     print(x)
                    index_houston.append(i)
                    day = x[1]
                    month = x[0]
                else:
                    day = x[0]
                    month = x[1]
    my_dates.append((day, month, year))
#     return day, month, year

In [67]:
day = [x[0] for x in my_dates]
month = [x[1] for x in my_dates]
year = [x[2] for x in my_dates]

In [68]:
def clean_year(x):
    x = re.sub(r'\D+','',x)
    if x == '':
        return None
    if len(x) == 2:
        x = '20' + str(x)
    if len(x) == 5:
        x= str(x)[:4]
#     x_final = re.sub(r'\D+', '', x)
    return x

In [69]:
def clean_day(x):
    if x == '':
        return None
    else:
        x = re.sub(r'\D+', '', x)
        return x

In [70]:
df['day'] = day
df['month'] = month
df['year'] = year

In [71]:
df['std_year'] = df['year'].apply(clean_year)
df['std_year'] = df['std_year'].astype(float)

In [72]:
df['day'] = df['day'].apply(clean_day)
df['std_day'] = df['day'].astype(float)

### Months standarization

In [73]:
months = {'1': ['jan', 'gen'],
         '2': [r'fe[bv]'],
         '3': [r'mar', r'macrh'],
         '4': [r'apr'],
         '5':[r'ma[ygi]'],
         '6':[r'jun', r'giug', r'juin'],
          '7':[r'jul', r'lug'],
          '8':[r'aug', 'ago', 'aot'],   #to mozna tez dac, ze po prstu ma sie zaczynac na a
         '9': [r'se[pt]', r'sptember'],
          '10':[r'o[ct]t'],
          '11':[r'nov'],
          '12':[r'd[ei]c'],
          '0': [r'rev', r'soook', r'ago', r'gtg', r'opened', r'cont']
         
         }

In [74]:
def std_month(x):
    x = x.strip()
    if x.isalpha():
#         my_month = ''
        for month, patterns in months.items():
            my_month = ''
            for pattern in patterns:
                if len(re.findall(pattern, x)) > 0:
                    my_month = month
                    return my_month
#                 elif len(re.findall(pattern, x)) == 0:
#                     return 'Error'
    
    elif x == '':
        return None
    
    elif x.isdigit() == False:
        return '0'
    else:
        return x

In [75]:
df['std_month'] = df['month'].apply(std_month)
df['std_month'] = df['std_month'].astype(float)

### Data Correction

In [76]:
warning_signes = []

for i, row in df.iterrows():
    
    # day_error
    if row['std_day'] == 0 or row['std_day'] > 31:
        is_warning = True
    
    # year error
    elif row['std_year'] > 2022 or row['std_year'] < 1995:
        is_warning = True
    else:
        is_warning = False
        

    if row['std_month'] < 1 or row['std_month'] > 12:
        is_warning = True
#     else:
#         is_warning = False
        
    warning_signes.append(is_warning)



df['Date_error'] = warning_signes

In [78]:
df['Date_error'].value_counts()

False    17536
True        22
Name: Date_error, dtype: int64

In [79]:
# ## Correction pipeline
# ## This was an attempt to correct Data errors, but since there are only few errors it's better to correct it manualy
# attempts = [1, 2, 3]
# for i in attempts:
#     final_dates = []

    
#     print(f'This is {i} iteration and we have such errors:')
#     print(df['Date_error'].value_counts())
    
#     for i, row in df.iterrows():
#         if row['Date_error'] == True:
#             if len(row['report_date']) > 1:
#                 final
#                 final_dates.append(row['report_date'][1])
#             elif len(row['report_date_2']) > 0:
#                 final_dates.append(row['report_date_2'][0])
#             elif len(row['date']) > 0:
#                 final_dates.append(row['date'][0])
#             else:
#                 final_dates.append('')
            
#     df_temp['final_dates'] = final_dates
#     df_temp['final_dates'] = df_temp['final_dates'].apply(final_dates_update)
#     df_temp['final_dates_test'] = df_temp['final_dates'].apply(final_dates_transform)
    
    
#     # this method include that in US month is first
#     index_houston = []
#     my_dates = []
#     for i, row in df_temp.iterrows():
#         x = row['final_dates_test']
#         day = ''
#         month = ''
#         year = ''
#         if len(x) == 3:
#             if len(x[0]) == 4 and x[0].isdigit() == True:
#                 year = x[0]
#                 month = x[1]
#                 day = x[2]
#             else:
#                 year = x[-1]

#                 # miesiac slownie:
#                 if x[1].isalpha() == True:
#                     day = x[0]
#                     month = x[1]

#                 elif x[0].isalpha() == True:
#                     day = x[1]
#                     month = x[0]

#                 # wieksza liczba od 12:
#                 elif x[0].isdigit() and int(x[0]) > 12:
#                     day = x[0]
#                     month = x[1]

#                    # wieksza liczba od 12:
#                 elif x[1].isdigit() and int(x[1]) > 12:
#                     day = x[1]
#                     month = x[0]


#                     #defaultowe wartosci
#                 else:
#                     if 'houston' in row['clean_text']:
#     #                     print(x)
#                         index_houston.append(i)
#                         day = x[1]
#                         month = x[0]
#                     else:
#                         day = x[0]
#                         month = x[1]
#         my_dates.append((day, month, year))
        
#     day = [x[0] for x in my_dates]
#     month = [x[1] for x in my_dates]
#     year = [x[2] for x in my_dates]
    
#     df_temp['day'] = day
#     df_temp['month'] = month
#     df_temp['year'] = year
    
#     df_temp['day'] = df_temp['day'].apply(clean_day)
#     df_temp['std_day'] = df_temp['day'].astype(float)
#     df_temp['std_year'] = df_temp['year'].apply(clean_year)
#     df_temp['std_year'] = df_temp['std_year'].astype(float)
#     df_temp['std_month'] = df_temp['month'].apply(std_month)
#     df_temp['std_month'] = df_temp['std_month'].astype(float)
#     warning_signes = []

#     for i, row in df_temp.iterrows():

#         # day_error
#         if row['std_day'] == 0 or row['std_day'] > 31:
#             is_warning = True

#         # year error
#         elif row['std_year'] > 2022 or row['std_year'] < 1995:
#             is_warning = True
#         else:
#             is_warning = False


#         if row['std_month'] < 1 or row['std_month'] > 12:
#             is_warning = True
#     #     else:
#     #         is_warning = False

#         warning_signes.append(is_warning)



#     df_temp['Date_error'] = warning_signes

This is 1 iteration and we have such errors:
False    17536
True        22
Name: Date_error, dtype: int64


NameError: name 'df_temp' is not defined

In [80]:
df_st = df[['name', 'source', 'clean_text', 'SN_checked_final', 'job', 'report_std_short', 'std_year', 'std_month', 'std_day']]

In [81]:
df_st.to_csv('Tagged_July_25_22.csv')